<a href="https://colab.research.google.com/github/aakhterov/ML_algorithms_from_scratch/blob/master/statistical_tests_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Various statistical tests from scratch

References

1. https://www.scribbr.com/statistics/t-test/
2. https://www.statisticshowto.com/probability-and-statistics/t-test/
3. https://en.wikipedia.org/wiki/Student%27s_t-test

> Indented block



In [70]:
import numpy as np
from sklearn.datasets import load_iris
from scipy.stats import t, ttest_ind, ttest_rel

In [15]:
alpha = 0.05 # significance level

## 1. T-test (Student's T-test)

### 1.1. Two sample T-test

Let's imagine we want to know whether the mean petal length of iris flowers differs according to their species. We're going to pick 50 petals length of versicolor and virginica species. Then we want to test the difference between these two groups using a T-test and null and alterative hypotheses.

**H0**: the true difference between these group means is zero.

**H1**: the true difference is different from zero.

The formulas for the $ t $ are:

1. In the case of equal sample sizes and variance:

$ t = \frac{\bar{x_1} - \bar{x_2}}{s \sqrt{\frac{2}{n}}} $

2. In the case of equal or unequal sample sizes and similar variances ($ \frac{1}{2} < \frac{s_{x_1}}{s_{x_2}} < 2 $):

$ t = \frac{\bar{x_1} - \bar{x_2}}{s_p \sqrt{\frac{1}{n_1} + \frac{1}{n_1}}} $,

where $ s_p = \sqrt{ \frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2} } $ is the pooled standard deviation

3. In the case of equal or unequal sample sizes and unequal variances ($
 s_{x_1} > 2s_{x_2} or s_{x_2} > 2s_{x_1} $ ) (Welch's t-test):

$ t = \frac{\bar{x_1} - \bar{x_2}}{s_\bar\Delta} $,

where $ s_\bar\Delta = \sqrt{(\frac{s_1}{\sqrt{n_1}})^2 + (\frac{s_2}{\sqrt{n_2}})^2} $

In [16]:
data = load_iris(as_frame=True)
df = data['data']
df['specie'] = list(map(lambda x: data['target_names'][x], data['target']))
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),specie
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [17]:
df['specie'].value_counts()

setosa        50
versicolor    50
virginica     50
Name: specie, dtype: int64

In [18]:
g1 = df.loc[df['specie'] == 'versicolor']['petal length (cm)'].to_numpy()
g2 = df.loc[df['specie'] == 'virginica']['petal length (cm)'].to_numpy()

In [19]:
s1, s2 = np.std(g1), np.std(g2)
1/2 < s1/s2 < 2

True

**Conclusion:** Hence, we have case of equal sample sizes and similar variances (#2)

In [20]:
n1, n2 = len(g1), len(g2)
mean1, mean2 = np.mean(g1), np.mean(g2)

In [21]:
s_p = np.sqrt(( (n1-1)*s1**2 + (n2-1)*s2**2 )/(n1 + n2 -2)) # calculate the pooled standard deviation
print(f"The pooled standard deviation:  {s_p}")

The pooled standard deviation:  0.5073933385451567


In [93]:
t_scratch = (mean1 - mean2) / (s_p * np.sqrt(1/n1 + 1/n2))
print(f"Two sample T-test: {t_scratch}")

Two sample T-test: -12.731739873689888


In [48]:
cv = t.ppf(alpha/2, n1 + n2 - 2) # calculate the critical values for the two-tailed test
print(f"The confidence intervals (CI) for DOF (degree if freedom) of {n1+n2-2} and significant level of {alpha}:\n"
      f"- left CI: from -inf to {cv}\n"
      f"- right CI: from {-cv} to inf\n")

The confidence intervals (CI) for DOF (degree if freedom) of 98 and significant level of 0.05:
- left CI: from -inf to -1.9844674544266925
- right CI: from 1.9844674544266925 to inf



In [49]:
t_scratch < cv

True

In [50]:
p_value = t.cdf(t_scratch, n1 + n2 -2)*2 # let's calculate p-value for the obtained T-test value
print(f"The probability of obtaining T-test results of {t_scratch} at least as extreme as the result actually observed, \n"
f"under the assumption that the H0 is correct equal to {p_value}")

The probability of obtaining T-test results of -12.731739873689888 at least as extreme as the result actually observed, 
under the assumption that the H0 is correct equal to 1.7129346742281333e-22


**Conclusion:** The value of T-test is in the left CI, p-value is less then 0.025 therefore we can reject HO => the true difference is different from zero.

In [26]:
# Let's use a function from scipy library

In [27]:
t_scipy = ttest_ind(g1, g2, equal_var=False)
t_scipy

Ttest_indResult(statistic=-12.603779441384987, pvalue=4.900287527398095e-22)

**Conclusion:** The result of the T-test obtained by in-built function differs from implementation from scratch. It looks a bit wired.  

### 1.2. Paired T-test

Let's assume we want to enlarge the petals of the iris specie of 'setosa'. We invented some treatment and applied it to irises and we want to figure out if our treatment is working or not. So, we have two group of the petal lengths: the first one is the petal lengths before applying the treatment and the other one is after.

**H0**: the second group mean isn't greater then the first group mean

**H1**: the second group mean is greater then the first group mean

$ t = \frac{\bar{X_D}}{s_D/\sqrt{n}} $,

where $ \bar{X_D} $ and $ s_D $ are the average and standard deviation of the differences between all pairs.

In [87]:
g1 = df.loc[df['specie'] == 'setosa']['petal length (cm)'].to_numpy()
g2 = g1 + np.random.normal(loc=1, scale=1, size=len(g1))

In [92]:
n = len(g1)
D = g2 - g1
X_D, s_D = np.mean(D), np.std(D)
print(f"The mean and standard deviation of the differences: {X_D}, {s_D}")

The mean and standard deviation of the differences: 1.193639849346733, 1.1004483817162092


In [94]:
t_scratch = X_D/(s_D / np.sqrt(n))
print(f"Paired T-test: {t_scratch}")

Paired T-test: 7.669881166540969


In [102]:
cv = t.ppf(1-alpha, n - 1) # calculate the critical values for the right-tailed test
print(f"A confidence interval (CI) for DOF (degree if freedom) of {n-1} and significant level of {alpha}:\n"
      f"from {cv} to inf\n")

A confidence interval (CI) for DOF (degree if freedom) of 49 and significant level of 0.05:
from 1.6765508919142629 to inf



In [103]:
t_scratch > cv

True

In [104]:
p_value = 1 - t.cdf(t_scratch, n - 1) # let's calculate p-value for the obtained T-test value
print(f"The probability of obtaining T-test results of {t_scratch} at least as extreme as the result actually observed, \n"
f"under the assumption that the H0 is correct equal to {p_value}")

The probability of obtaining T-test results of 7.669881166540969 at least as extreme as the result actually observed, 
under the assumption that the H0 is correct equal to 3.0641400527997575e-10


**Conclusion:** The value of T-test is in CI, p-value is less then 0.05 therefore we can reject HO => the second group mean is greater then the first group mean

In [ ]:
# Let's use a function from scipy library

In [91]:
t_scipy = ttest_rel(g2, g1, alternative='greater')
t_scipy

TtestResult(statistic=7.592794977258548, pvalue=4.0263921423052266e-10, df=49)

**Conclusion:** The result of the T-test obtained by in-built function differs from implementation from scratch as in the case of two sample T-test. It also looks wired.  

### 1.3. One-sample T-test